In [ ]:
import numpy as np
from scipy.stats import poisson, beta, lognorm

# Parâmetros por tranche
tranches = [
    {"n": 50, "p": 0.03, "mu_rec": 0.4, "sigma_rec": 0.1, "mu_L": 1000, "sigma_L": 300, "mu_T": 5, "sigma_T": 1},  # Tranche 1
    {"n": 30, "p": 0.05, "mu_rec": 0.5, "sigma_rec": 0.15, "mu_L": 2000, "sigma_L": 600, "mu_T": 7, "sigma_T": 2}, # Tranche 2
    {"n": 20, "p": 0.08, "mu_rec": 0.6, "sigma_rec": 0.2, "mu_L": 500, "sigma_L": 150, "mu_T": 3, "sigma_T": 0.5},  # Tranche 3
]

simulations = 1000  # Número de simulações
r = 0.05  # Taxa de desconto anual
j = 0.1   # Taxa de juros anual dos empréstimos
max_years = 10  # Período máximo de análise (horizonte)

# Função para calcular alpha e beta da distribuição Beta
def beta_params(mean, var):
    alpha = mean * ((mean * (1 - mean) / var) - 1)
    beta = (1 - mean) * ((mean * (1 - mean) / var) - 1)
    return alpha, beta

# Função para calcular os parâmetros da lognormal
def lognormal_params(mean, std):
    var = std**2
    mu = np.log(mean**2 / np.sqrt(var + mean**2))
    sigma = np.sqrt(np.log(1 + var / mean**2))
    return mu, sigma

# Simulação Monte Carlo
all_cashflows = []

for _ in range(simulations):
    annual_cashflows = np.zeros(max_years)  # Fluxos anuais da carteira
    for tranche in tranches:
        # Parâmetros da tranche
        n, p = tranche["n"], tranche["p"]
        mu_rec, sigma_rec = tranche["mu_rec"], tranche["sigma_rec"]
        mu_L, sigma_L = tranche["mu_L"], tranche["sigma_L"]
        mu_T, sigma_T = tranche["mu_T"], tranche["sigma_T"]
        
        # Gerar valores dos empréstimos (Lognormal)
        mu, sigma = lognormal_params(mu_L, sigma_L)
        loan_values = lognorm.rvs(s=sigma, scale=np.exp(mu), size=n)
        
        # Gerar maturidades (Normal truncada)
        maturities = np.clip(np.random.normal(mu_T, sigma_T, n), 1, max_years).astype(int)
        
        # Determinar número de defaults (Poisson)
        lambda_k = n * p
        D_k = poisson.rvs(lambda_k)
        
        # Empréstimos que entram em default
        default_indices = np.random.choice(n, size=D_k, replace=False, p=None)
        alpha_k, beta_k = beta_params(mu_rec, sigma_rec**2)
        recoveries = beta.rvs(alpha_k, beta_k, size=D_k)
        
        for i in default_indices:
            L = loan_values[i]
            T_i = maturities[i]
            A = L / T_i
            balance = L
            for t in range(T_i):
                if t >= max_years: break  # Parar se o tempo excede o horizonte de análise

                # Verificar default
                if t == T_i - 1 or np.random.uniform(0, 1) < p:  # Default no período
                    recovery = recoveries[np.where(default_indices == i)[0][0]] * balance
                    annual_cashflows[t] += recovery  # Adicionar recuperação
                    break
                else:
                    # Calcular juros e parcela total
                    J_t = balance * j
                    P_t = A + J_t

                    # Adicionar fluxo regular ao ano atual
                    annual_cashflows[t] += P_t

                    # Atualizar saldo devedor
                    balance -= A
        
        # Empréstimos que não entram em default
        non_default_indices = set(range(n)) - set(default_indices)
        for i in non_default_indices:
            L = loan_values[i]
            T_i = maturities[i]
            A = L / T_i
            balance = L
            for t in range(T_i):
                if t >= max_years: break  # Parar se o tempo excede o horizonte de análise
                
                # Calcular juros e parcela total
                J_t = balance * j
                P_t = A + J_t
                
                # Adicionar fluxo regular ao ano atual
                annual_cashflows[t] += P_t
                
                # Atualizar saldo devedor
                balance -= A
    
    all_cashflows.append(annual_cashflows)

# Cálculo do valor esperado dos fluxos por ano
expected_cashflows = np.mean(all_cashflows, axis=0)

# Cálculo do valor presente dos fluxos
present_value = np.sum(expected_cashflows / (1 + r) ** np.arange(1, max_years + 1))

# Resultados
print("Fluxos esperados por ano:")
for t, cashflow in enumerate(expected_cashflows, start=1):
    print(f"Ano {t}: {cashflow:.2f}")
    
print(f"\nValor Presente da Carteira: {present_value:.2f}")
